# Laboratorio de regresión logística

|                |   |
:----------------|---|
| **Nombre**     | Morales Ramirez Valadez Juan Alvaro  |
| **Fecha**      |  07/10/2025 |
| **Expediente** | 745903  | 

In machine learning, Support Vector Machines (SVM) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. It is mostly used in classification problems. In this algorithm, each data item is plotted as a point in p-dimensional space (where p is the number of features), with the value of each feature being the value of a particular coordinate. Then, classification is performed by finding the hyper-plane that best differentiates the two classes (or more if we have a multi class problem):

$$ f(x) = w^T \varphi(x) + b $$

where $\varphi: X \rightarrow F $ is a function that makes each input point $x$ correspond to a point in F, where F is a Hilbert space.

In addition to performing linear classification, SVMs can efficiently perform a non-linear classification, implicitly mapping their inputs into high-dimensional feature spaces (more specifically using the kernel trick, like the RBF funcion). 

[1]

OLS utilizes the squared residuals to fit the parameters. Large residuals caused by outliers may worsen the accuracy significantly.

Support Vectors use piecewise linear functions to counter this, in which a hyperparameter  $\epsilon$ called the margin lets errors that are less or equal to it be 0, and error larger than it be $e - \epsilon$. 

The problem to solve is:

\begin{split}
        \min_{w, b, \xi, \xi^*} \mathcal{P}_\epsilon(w, b, \xi) &= \frac{1}{2} w^T w + c \sum_{k=1}^{N} \xi_k \\
        \text{s.t. } & y_k [w^T \varphi(x_k) - b] \geq 1- \xi_k,\ \ k = 1, ..., N \\
        & \xi_k \geq 0,\ \ k = 1, ..., N
\end{split}


The most important question that arises when using a SVM is how to choose the correct hyperplane. Consider the following scenarios:

### Scenario 1

In this scenario there are three hyperplanes called A, B, and C. Now, the problem is to identify the hyperplane which best differentiates the stars and the circles.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_21-2.png" alt="what image shows"></center>

In this case, hyperplane B separates the stars and the circle betters, hence it is the correct hyperplane.


### Scenario 2

Now take another scenario where all three hyperplanes are segregating classes well. The question that arises is how to choose the best hyperplane in this situation.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_4-2.png" alt="what image shows"></center>

In such scenarios, we calculate the margin (which is the distance between nearest data point and the hyperplane). The hyperplane with the largest margin will be considered as the correct hyperplane to classify the dataset.

Here C has the largest margin. Hence, it is considered as the best hyperplane.


### Kernels
Knowing 
$$ w = \sum_{k=1}^{N} \alpha_k y_k \varphi(x_k) $$

And
$$ y_{pred} = w^T \varphi(x) + b $$

Then 
$$ y_{pred} = (\sum_{k=1}^{N} \alpha_k y_k \varphi(x_k))^T \varphi(x) + b $$

Where $\varphi$ is a function that makes each input in $x$ correspond to a point in $F$ (a Hilbert space). This can be seen as processing and transforming the input featuers to keep the model's convexity. [2]

This also allows us to transform the inputs into another space where they might be more easily classified.

<center><img src=https://miro.medium.com/max/838/1*gXvhD4IomaC9Jb37tzDUVg.png alt="what image shows"></center>

## ROC and AUC

A ROC (Receiver Operating Characteristic) is a graph that shows how the classification model performs at the classification thresholds. 

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis. This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a larger area under the curve (AUC) is usually better. [3]

True Positive Rate is a synonym for Recall and defined as:
$$ TPR = \frac{TP}{TP + FN} $$

False Positive Rate is a synonym for Specificity and defined as:

$$ FPR = \frac{FP}{FP + TN} $$

ROC curves are typically used in binary classification to study the output of a classifier. In order to extend ROC curve and ROC area to multi-label classification, it is necessary to binarize the output. One ROC curve can be drawn per label, but one can also draw a ROC curve by considering each element of the label indicator matrix as a binary prediction (micro-averaging).

E.g. If you lower a classification threshold, more items would be classified as positive, increasing False Positives and True Positives.

AUC stands for Area under the ROC.

## Ejercicio 1

- Utiliza el dataset `Iris`, modela con SVC y haz Cross-Validation de diferentes kernels ('linear', 'poly', 'rbf', 'sigmoid').
- Modela con LogisticRegression.
- El método de Cross-Validation es K-Folds con $k=10$.
- Utiliza el AUC como métrico de Cross-Validation.
- Compara resultados.

In [27]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd


In [29]:
# Cargar el dataset Iris
X, y = load_iris(return_X_y=True)

# Modelos a evaluar
models = [
    ('SVC linear', SVC(kernel='linear', probability=True)),
    ('SVC poly', SVC(kernel='poly', degree=2, probability=True)),
    ('SVC rbf', SVC(kernel='rbf', probability=True)),
    ('SVC sigmoid', SVC(kernel='sigmoid', probability=True)),
    ('LogisticRegression', LogisticRegression(max_iter=200))
]

# Cross-validation con 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Guardar resultados
results = []

for name, model in models:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', model)
    ])

    auc_scores = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        pipeline.fit(X_train, y_train)
        y_prob = pipeline.predict_proba(X_test)

        score = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
        auc_scores.append(score)

    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)

    results.append({
        'Modelo': name,
        'AUC promedio': round(mean_auc, 4),
        'AUC std': round(std_auc, 4)
    })

In [30]:
# Mostrar tabla de resultados
df_resultados = pd.DataFrame(results).set_index("Modelo")
df_resultados.head()


,AUC promedio,AUC std
Modelo,,
SVC linear,0.9967,0.0068
SVC poly,0.9547,0.0579
SVC rbf,0.9987,0.0040
SVC sigmoid,0.9720,0.0290
LogisticRegression,0.9980,0.0060


En este trabajo se abordó un problema de clasificación utilizando el dataset Iris. El objetivo fue predecir la especie de una flor (setosa, versicolor o virginica) a partir de cuatro variables numéricas que describen sus características morfológicas: longitud y ancho del sépalo, y longitud y ancho del pétalo.

Se aplicaron cinco modelos de clasificación: SVC con cuatro tipos de kernel (lineal, polinomial de grado 2, radial RBF y sigmoide) y regresión logística. Todos los modelos se evaluaron usando validación cruzada con k=10 (k-fold) y la métrica utilizada fue el AUC, que permite medir la capacidad del modelo para distinguir entre clases.

Los resultados mostraron que el SVC con kernel RBF obtuvo el mejor desempeño, con un AUC promedio de 0.9987, seguido muy de cerca por la regresión logística con 0.9980 y el SVC lineal con 0.9967. El modelo con kernel sigmoide logró un AUC de 0.9727, mientras que el kernel polinomial fue el menos efectivo con un AUC de 0.9560 y mayor variabilidad entre pliegues.

Estos resultados indican que el problema de clasificación en el dataset Iris puede resolverse de forma muy efectiva con modelos tanto lineales como no lineales, siendo el kernel RBF y la regresión logística las opciones más precisas y estables en este caso.

## Ejercicio 2
- Repite el ejercicio 1 con el dataset `Default`. Utiliza `default` como target.

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

df_default = pd.read_csv('Default.csv') 
df_default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [15]:
# Convertir categoricas a numericas (dummies)
df_default['default'] = df_default['default'].str.strip().str.lower().map({'no': 0, 'yes': 1})
df_default['student'] = df_default['student'].str.strip().str.lower().map({'no': 0, 'yes': 1})

# Separar variables y target
X = df_default.drop(columns=['default'])
y = df_default['default']


# Modelos
models = [
    ('SVC linear', SVC(kernel='linear', probability=True)),
    ('SVC poly', SVC(kernel='poly', degree=2, probability=True)),
    ('SVC rbf', SVC(kernel='rbf', probability=True)),
    ('SVC sigmoid', SVC(kernel='sigmoid', probability=True)),
    ('LogisticRegression', LogisticRegression(max_iter=200))
]

# Cross-validation con 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Guardar resultados
results = []

for name, model in models:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', model)
    ])

    auc_scores = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        pipeline.fit(X_train, y_train)
        y_prob = pipeline.predict_proba(X_test)[:, 1]  # Probabilidades para la clase 1 (default = sí)

        score = roc_auc_score(y_test, y_prob)
        auc_scores.append(score)

    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)

    results.append({
        'Modelo': name,
        'AUC promedio': round(mean_auc, 4),
        'AUC std': round(std_auc, 4)
    })

In [25]:
# Mostrar tabla de resultados
df_resultados = pd.DataFrame(results).set_index("Modelo")
df_resultados.head()


,AUC promedio,AUC std
Modelo,,
SVC linear,0.6672,0.3944
SVC poly,0.8875,0.0412
SVC rbf,0.8420,0.0400
SVC sigmoid,0.7291,0.0450
LogisticRegression,0.9485,0.0221


Los resultados que obtuve muestran claramente que la regresión logística es el modelo que mejor desempeño ofrece para predecir el default, con un AUC promedio de 0.9485 y una desviación estándar muy baja, lo que indica estabilidad y consistencia en sus predicciones; aunque algunos SVC con kernels polinomial y RBF también funcionaron bien, ninguno superó la eficacia y confiabilidad de la regresión logística, mientras que otros kernels como el lineal y sigmoid demostraron ser menos efectivos y más variables.

# Addendum

Métricos disponibles para clasificación:
- ‘accuracy’
- ‘balanced_accuracy’
- ‘top_k_accuracy’
- ‘average_precision’
- ‘neg_brier_score’
- ‘f1’
- ‘f1_micro’
- ‘f1_macro’
- ‘f1_weighted’
- ‘f1_samples’
- ‘neg_log_loss’
- ‘precision’ etc.
- ‘recall’ etc.
- ‘jaccard’ etc.
- ‘roc_auc’
- ‘roc_auc_ovr’
- ‘roc_auc_ovo’
- ‘roc_auc_ovr_weighted’
- ‘roc_auc_ovo_weighted’
- ‘d2_log_loss_score’

# References

[1] Shigeo Abe.Support Vector Machines for Pattern Classification,2Ed.Springer-Verlag London,2010. ISBN978-1-84996-097-7. URLhttps://www.springer.com/gp/book/9781849960977.

[2] Johan A K Suykens, Tony Van Gestel, Jos De Brabanter, BartDe Moor, and Joos Vandewalle.Least Squares Support VectorMachines. World Scientific,2002. ISBN9789812381514. URLhttps://www.worldscientific.com/worldscibooks/10.1142/5089.

[3] Bradley, A. P. (1997). The use of the area under the ROC curve in the evaluation of machine learning algorithms. Pattern recognition, 30(7), 1145-1159. URL https://www.researchgate.net/post/how_can_I_interpret_the_ROC_curve_result